# Building a Spotify Recommendation Engine for Music Labels
## Data Collection, Cleaning, and EDA
### Flatiron School Data Science Program Phase 4 Project<br>Justin Williams & Khyatee Desai

In [100]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Webscraping

## Sub Pop Records Artists

In [54]:
# retrieve html page and create beautifulsoup object

page = requests.get('https://www.subpop.com/artists/list')
soup = BeautifulSoup(page.text)
soup.prettify

# parse through soup and save artist names in a list
subpop = []
artists = soup.find(id="all-artists")
for name in artists.findAll('a')[:-1]:
    subpop.append(name.text)
subpop

## 4AD Artists

In [58]:
# retrieve html page and create beautifulsoup object
page = requests.get('https://4ad.com/artists')
soup = BeautifulSoup(page.text)
soup.prettify

# parse through soup and save artist names in a list
AD4 = []
artists = soup.find(id="artistlistingmain")
for name in artists.findAll('a',href=True):
    AD4.append(name.text)
AD4

['50 Foot Wave',
 'A.R. Kane',
 'Adrianne Lenker',
 'Air Miami',
 'Aldous Harding',
 'The Amps',
 'Anni Rossi',
 'Ariel Pink',
 "Ariel Pink's Haunted Graffiti",
 'Atlas Sound',
 'Barbara Carlotti',
 'Bauhaus',
 'Bearz',
 'Becky and the Birds',
 'bEEdEEgEE',
 'Beirut',
 'Belly',
 'Bettie Serveert',
 'The Big Pink',
 'Big Thief',
 'Bing & Ruth',
 'The Birthday Party',
 'The Birthday Party/Lydia Lunch',
 'Blonde Redhead',
 'Blood Diamonds',
 'Bon Iver',
 'The Breeders',
 'Brendan Perry',
 'Broken Records',
 'The Bulgarian State Radio & Television Female Vocal Choir',
 'Camera Obscura',
 'Cass McCombs',
 'Celebration',
 'Clan Of Xymox',
 'Cocteau Twins',
 'Cocteau Twins And Harold Budd',
 'Colin Newman',
 'Colourbox',
 'Cuba',
 'Cupol',
 'D.D Dumbo',
 'Dance Chapter',
 'Daughter',
 'Daughter|Warpaint',
 'David Byrne & St. Vincent',
 'Dead Can Dance',
 'Deerhunter',
 'Department Of Eagles',
 'Diana Gordon',
 'Dif Juz',
 'Dry Cleaning',
 'Efterklang',
 'EL VY',
 'Electricity In Our Homes',
 

## Fat Possum Records

In [66]:
# retrieve html page and create beautifulsoup object
page = requests.get('https://fatpossum.com/featured-artists/all-artists/')
soup = BeautifulSoup(page.text)
soup.prettify

# # parse through soup and save artist names in a list
possum = []
artists = soup.find(id="all-artists")
for name in artists.findAll('a',href=True, title=True):
    possum.append(name.text)
possum

['2:54',
 'A. A. Bondy',
 'Adam Green',
 'Adam Torres',
 'Al Green',
 'American Wrestlers',
 'Andrew Bird',
 'Ann Peebles',
 'Asie Payton',
 'Band of Horses',
 'Bash & Pop',
 'Bass Drum of Death',
 'Beaten Awake',
 'Big Harp',
 'Black Bananas',
 'The Black Keys',
 'Blaze Foley',
 'Bob Log III',
 'Camu Tao',
 'Cassowary',
 'Caveman',
 'Cecil Barfield',
 'Cedell Davis',
 'Charles Caldwell',
 'Christopher Owens',
 'Colour Revolt',
 'Communions',
 'Country Westerns',
 'Courtney Marie Andrews',
 'Crocodiles',
 'Dax Riggs',
 'Day Wave',
 'Dead Fingers',
 'Designer Records',
 'Digital Leather',
 'Dilly Dally',
 'Dinosaur Jr.',
 'The Districts',
 'Don Bryant',
 'The Dove & The Wolf',
 'Dralms',
 'El-P',
 'Ellis',
 'Elmo Williams & Hezekiah Early',
 'Empath',
 'Famous L. Renfroe',
 'Fat White Family',
 'The Felice Brothers',
 'Francisco The Man',
 'Frankie Rose',
 'Fred McDowell',
 'Friends',
 'Furry Lewis',
 'Futurebirds',
 'George Mitchell Collection',
 'Gold Connections',
 'Gross Magic',
 'H

## Matador Records

In [85]:
# retrieve html page and create beautifulsoup object
page = requests.get('https://www.matadorrecords.com/artists')
soup = BeautifulSoup(page.text)
soup.prettify

matador = []
artists = soup.find(class_="artists-page")
for name in artists.findAll('a',href=True):
    if name.text:
        matador.append(name.text) 

# get alumni artists
alumns = soup.find(class_="col-section")
for name in alumns.findAll('a',href=True)[1:]:
    matador.append(name.text)
matador

['Algiers',
 'Belle and Sebastian',
 'Body/Head',
 'Car Seat Headrest',
 'Courtney Barnett & Kurt Vile ',
 'DARKSIDE',
 'Desert Sessions',
 'Gang of Four',
 'Iceage',
 'Interpol',
 'Julien Baker',
 'Kim Gordon',
 'King Krule',
 'Kurt Vile',
 'Lucy Dacus',
 'Mdou Moctar',
 'Muzz',
 'Pavement',
 'Perfume Genius',
 'Queens of the Stone Age',
 'Savages',
 'Snail Mail',
 'Spoon',
 'Stephen Malkmus',
 'Stephen Malkmus & The Jicks',
 'Steve Gunn',
 'Yo La Tengo',
 '18th Dye',
 '2 Foot Flame',
 'AC Newman',
 'Aereogramme',
 'Antonia Sellbach',
 'Arab Strap',
 'Arsonists',
 'Arsonists/Non Phixion (Skeme Team)',
 'Babylon Dance Band',
 'Bailter Space',
 'Barbara Manning',
 'Bardo Pond',
 'Bardo Pond/Mogwai',
 'Bassholes',
 'Bettie Serveert',
 'The Bionaut',
 'Bits Of Shit',
 'Blair Kelly',
 'Boards of Canada',
 'Brightblack Morning Light',
 'Bullet Lavolta',
 'Bunnybrains',
 'burger/ink',
 'Cat Power',
 'The Cave Singers',
 'Ceremony',
 'Chain Gang',
 'Chavez',
 'Chelsea Light Moving',
 'Chris K

## Third Man Records

In [90]:
# retrieve html page and create beautifulsoup object
page = requests.get('https://thirdmanstore.com/bands')
soup = BeautifulSoup(page.text)
soup.prettify


# parse through soup and save artist names in a list
thirdman = []
artists = soup.find(class_="category-view")
for name in artists.findAll('a',href=True):
    thirdman.append(name.text)
thirdman

['The 5.6.7.8’s',
 'Aaron Lee Tasjan',
 'ADULT.',
 'AJJ',
 'Alabama Shakes',
 'Albert Ayler',
 'Algebra Mothers',
 'Alison Mosshart',
 'Alloy Orchestra',
 'Amy Walker',
 'Archie & The Bunkers',
 'Ashley Monroe',
 'Autolux',
 'Aziz Ansari',
 'Barrett Strong',
 'Beck',
 'Benjamin Booker',
 'Big Joanie',
 'Bill Burr',
 'Bill Callahan',
 'Billy Wayne Davis',
 'The Black Belles',
 'Black Milk',
 'The Blind Shake',
 'Blind Willie Johnson',
 'Blind Willie McTell',
 'Blitzen Trapper',
 'Bob Seger',
 'Bonny Doon',
 'Boris',
 'BP Fallon',
 'Brad Wenzel',
 'Brendan Benson',
 'Brittany Howard & Ruby Amanfu',
 'Broken Social Scene',
 'Caddywhompus',
 'Caetano Veloso',
 'Captain Beefheart',
 'Carl Perkins',
 'Carl Sagan',
 'The Carter Family',
 'Chain and the Gang',
 'Chandra',
 'Charley Patton',
 'Cheap Time',
 'Chico Leverett',
 'Chris Thile and Michael Daves',
 'Chuck Berry',
 'Coldplay',
 'Cold War Kids',
 'Conan O’Brien',
 'Coupler',
 'Courtney Barnett',
 'Craig Brown Band',
 'D. A. Hunt',
 'Da

## XL Records

In [93]:
# retrieve html page and create beautifulsoup object
page = requests.get('https://xlrecordings.com/buy')
soup = BeautifulSoup(page.text)
soup.prettify


# parse through soup and save artist names in a list
XL = []
artists = soup.find(class_="artists")
for name in artists.findAll('a',href=True):
    XL.append(name.text)
XL

['Adele',
 'Arca',
 'Archy Marshall',
 'Atoms For Peace',
 'Baba Stiltz',
 'Badly Drawn Boy',
 'Basement Jaxx',
 'Batu',
 'Beck',
 'Bobby Womack',
 'Breakbeat Era',
 'Dark0',
 'Devendra Banhart',
 'Dizzee Rascal',
 'East India Youth',
 'Everything Is Recorded',
 'Fabiana Palladino',
 'GILA',
 'Gil Scott-Heron',
 'Gil Scott-Heron & Jamie xx',
 'Gotan Project',
 'HIRA',
 'Hugo Massien',
 'Ibeyi',
 'Jack Penate',
 'Jack White',
 'Jai Paul',
 'Jonny Greenwood',
 'Jonny L',
 'Joy Orbison & Overmono',
 'Jungle',
 'Kaytranada',
 'King Krule',
 'Kurupt FM',
 'Lapsley',
 'Leila',
 'Liss',
 'M.I.A.',
 'MssingNo',
 'New Gen',
 'Nines',
 'Novelist x Mumdance',
 'Overmono',
 'Peaches',
 'Powell',
 'Powell Tillmans',
 'REINEN',
 'Radiohead',
 'Ratatat',
 'Rostam',
 'Ruthven',
 'Sam Gellaitry',
 'Shamir',
 'Sigur Rós',
 'Since 1989',
 'Smerz',
 'Special Request',
 'The Avalanches',
 'The Higher',
 'The Horrors',
 'The Prodigy',
 'The Raconteurs',
 'The White Stripes',
 'Thom Yorke',
 'Titus Andronicu

## Dead Oceans

In [99]:
# retrieve html page and create beautifulsoup object (dead oceans website blocks requests, so using Wikipedia)
page = requests.get('https://en.wikipedia.org/wiki/Dead_Oceans#Artists')
soup = BeautifulSoup(page.text)
soup.prettify

# parse through soup and save artist names in a list
deadoceans = []
artists = soup.find(class_="div-col columns column-width")
for name in artists.findAll('a',href=True):
    deadoceans.append(name.text)
deadoceans

['A Place to Bury Strangers',
 'Akron/Family',
 'Julianna Barwick',
 'Bear in Heaven',
 'Better Oblivion Community Center',
 'Bill Fay',
 'Bishop Allen',
 'Bleached',
 'Bowerbirds',
 'Brazos',
 'Bright Eyes',
 'Califone',
 'Citay',
 'Destroyer',
 'Dirty Projectors',
 'Dub Thompson',
 'The Donkeys',
 'Evangelicals',
 'The Explorers Club',
 'Frog Eyes',
 'Gauntlet Hair',
 'Japanese Breakfast',
 'Khruangbin',
 'Alex Lahey',
 'The Luyas',
 'Mark McGuire',
 'Mitski',
 'Kevin Morby',
 'Mt. St. Helens Vietnam Band',
 'Nurses',
 'On Fillmore',
 'Phoebe Bridgers',
 'Phosphorescent',
 'Tom Rogerson',
 'Shame',
 'Slowdive',
 'Strand of Oaks',
 'Kane Strang',
 'Sun Airway',
 'The Tallest Man on Earth',
 'These Are Powers',
 'John Vanderslice',
 'Ryley Walker',
 'White Hinterland',
 'Marlon Williams']